# Claims Performance Analysis (Multi-File) — With Date Range Filter

This notebook ingests multiple **dealer claims** workbooks and performs the same analysis for each, plus a **combined view**.
It now includes a **date range filter** you can apply to **Created Date**, **Completed Date**, or **Either**.


## Inputs
- Default files (edit in **Parameters**):
  - `Claims.xlsx`
  - `Chrysler Claims.xlsx`
- Expected sheet: `Tasks`
- Expected columns: `Bucket Name`, `Labels`, `Created Date`, `Completed Date`


In [ ]:
# %%
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)


In [ ]:
# %%
# ----------------------------
# Parameters (EDITABLE)
# ----------------------------
INPUT_FILES = [
    {"name": "Claims", "filename": "Claims.xlsx"},
    {"name": "Chrysler Claims", "filename": "Chrysler Claims.xlsx"},
]

SHEET_NAME = "Tasks"
OUTPUT_XLSX = Path("Claims_Performance_Summary_ALL.xlsx")

# Warehouses for by-warehouse Pareto (edit as needed)
WAREHOUSES = ["OKC", "Atlanta", "Orlando", "Ontario", "El Paso", "Flowood", "Phoenix", "Charlotte"]

# Max labels to show in each Pareto chart
TOP_N_LABELS = 15  # None to show all

# ----------------------------
# Date Range Filter
# ----------------------------
# Filter mode: "Created", "Completed", or "Either"
DATE_FILTER_MODE = "Created"

# Provide dates as 'YYYY-MM-DD' strings or None to disable.
# If only START_DATE is provided, filter from that date forward (inclusive).
# If only END_DATE is provided, filter up to that date (inclusive).
START_DATE = None      # e.g., "2024-01-01"
END_DATE = None        # e.g., "2024-12-31"


In [ ]:
# %%
# ----------------------------
# Helper functions
# ----------------------------
def resolve_path(fname: str) -> Path:
    p = Path(fname)
    if p.exists():
        return p
    fallback = Path("/mnt/data") / fname
    if fallback.exists():
        return fallback
    raise FileNotFoundError(f"Could not find input Excel '{fname}' at {p.resolve()} or {fallback.resolve()}")

def load_and_prepare(path: Path, sheet: str, dataset_name: str) -> pd.DataFrame:
    xls = pd.ExcelFile(path)
    if sheet not in xls.sheet_names:
        raise ValueError(f"Sheet '{sheet}' not found in {path.name}. Available: {xls.sheet_names}")
    df = pd.read_excel(path, sheet_name=sheet)

    required = ["Bucket Name", "Labels", "Created Date", "Completed Date"]
    for col in required:
        if col not in df.columns:
            raise ValueError(f"Expected column '{col}' not found in {path.name}/{sheet}")
    df["Bucket Name"] = df["Bucket Name"].astype(str).str.strip()
    df["Labels"] = df["Labels"].astype(str).str.strip()
    df["Created Date"] = pd.to_datetime(df["Created Date"], errors="coerce")
    df["Completed Date"] = pd.to_datetime(df["Completed Date"], errors="coerce")

    # Cycle time
    df["Cycle Time (Days)"] = (df["Completed Date"] - df["Created Date"]).dt.days

    # Dataset tag
    df["Dataset"] = dataset_name
    return df

def apply_date_filter(df: pd.DataFrame, mode: str, start_str, end_str) -> pd.DataFrame:
    """Filter rows by Created/Completed/Either date within [start, end] inclusive.
    - mode: 'Created', 'Completed', or 'Either'
    - start_str/end_str: 'YYYY-MM-DD' or None
    """
    if not start_str and not end_str:
        return df.copy()

    start = pd.to_datetime(start_str) if start_str else None
    end = pd.to_datetime(end_str) if end_str else None

    created = df["Created Date"]
    completed = df["Completed Date"]

    def in_range(series):
        mask = pd.Series(True, index=series.index)
        if start is not None:
            mask &= series >= start
        if end is not None:
            # inclusive upper bound
            mask &= series <= end
        return mask

    if mode.lower().startswith("created"):
        mask = in_range(created)
    elif mode.lower().startswith("completed"):
        mask = in_range(completed)
    else:  # 'Either' — created in range OR completed in range
        mask = in_range(created)
        mask |= in_range(completed)
    return df[mask].copy()

def explode_labels(df: pd.DataFrame) -> pd.DataFrame:
    out = df[["Dataset", "Bucket Name", "Labels"]].copy()
    out = out.assign(Labels=out["Labels"].str.split(";")).explode("Labels")
    out["Labels"] = out["Labels"].astype(str).str.strip()
    out = out[out["Labels"].ne("")]
    return out

def compute_bucket_perf(df: pd.DataFrame) -> pd.DataFrame:
    agg = (
        df.groupby(["Dataset", "Bucket Name"])["Cycle Time (Days)"]
          .agg(["count", "mean", "median", "min", "max"])
          .reset_index()
    )
    agg.rename(columns={
        "count": "Count",
        "mean": "Mean (days)",
        "median": "Median (days)",
        "min": "Min (days)",
        "max": "Max (days)"
    }, inplace=True)
    agg["Mean (days)"] = agg["Mean (days)"].round(2)
    agg["Median (days)"] = agg["Median (days)"].round(2)
    return agg

def compute_overall_pareto(labels_exploded: pd.DataFrame) -> pd.DataFrame:
    vc = labels_exploded["Labels"].value_counts().rename_axis("Label").reset_index(name="Count")
    total = vc["Count"].sum()
    vc["Cumulative %"] = (vc["Count"].cumsum() / total * 100).round(2)
    vc.insert(0, "Dataset", "ALL")
    return vc

def compute_by_wh_pareto(labels_exploded: pd.DataFrame, warehouses: list[str]) -> pd.DataFrame:
    wh = labels_exploded[labels_exploded["Bucket Name"].isin(warehouses)].copy()
    wh.rename(columns={"Bucket Name": "Warehouse"}, inplace=True)
    counts = (
        wh.groupby(["Dataset", "Warehouse", "Labels"])
          .size()
          .reset_index(name="Count")
    )
    counts["Percent"] = counts.groupby(["Dataset", "Warehouse"])["Count"].transform(lambda x: x / x.sum() * 100).round(2)
    counts = counts.sort_values(["Dataset", "Warehouse", "Count"], ascending=[True, True, False])
    counts["Cumulative %"] = counts.groupby(["Dataset", "Warehouse"])["Count"].transform(lambda x: (x.cumsum() / x.sum() * 100)).round(2)
    return counts

def plot_pareto(sub: pd.DataFrame, title: str, top_n=None):
    sub_sorted = sub.sort_values("Count", ascending=False)
    if top_n is not None:
        sub_sorted = sub_sorted.head(int(top_n))

    labels = sub_sorted["Labels"].tolist()
    counts = sub_sorted["Count"].tolist()
    cum_pct = sub_sorted["Cumulative %"].tolist()

    fig = plt.figure(figsize=(10, 5))  # one chart per figure
    ax1 = plt.gca()
    ax1.bar(labels, counts)
    ax1.set_xlabel("Label")
    ax1.set_ylabel("Count")
    ax1.set_title(title)

    ax2 = ax1.twinx()
    ax2.plot(range(len(labels)), cum_pct, marker="o")
    ax2.set_ylabel("Cumulative %")
    ax2.set_ylim(0, 100)

    ax1.tick_params(axis="x", rotation=45, labelsize=9)
    fig.tight_layout()
    plt.show()


In [ ]:
# %%
# ----------------------------
# Load & combine datasets
# ----------------------------
frames = []
missing = []
for item in INPUT_FILES:
    try:
        p = resolve_path(item["filename"])
        df_i = load_and_prepare(p, SHEET_NAME, dataset_name=item["name"])
        frames.append(df_i)
        print(f"Loaded {len(df_i):,} rows from {item['filename']} as dataset '{item['name']}'")
    except Exception as e:
        missing.append((item["filename"], str(e)))
        print(f"WARNING: Skipping '{item['filename']}' -> {e}")

if not frames:
    raise RuntimeError("No datasets loaded. Ensure input files exist.")

df_all = pd.concat(frames, ignore_index=True)
print(f"Before date filter: {len(df_all):,} rows")

In [ ]:
# %%
# ----------------------------
# Apply date filter
# ----------------------------
df_filtered = apply_date_filter(df_all, DATE_FILTER_MODE, START_DATE, END_DATE)
print(f"After date filter ({DATE_FILTER_MODE}, {START_DATE} to {END_DATE}): {len(df_filtered):,} rows")
df_filtered.head(3)

In [ ]:
# %%
# ----------------------------
# Bucket performance (filtered)
# ----------------------------
bucket_perf_by_dataset = compute_bucket_perf(df_filtered)

# Combined ALL view
df_filtered_all = df_filtered.copy()
df_filtered_all["Dataset"] = "ALL"
bucket_perf_all = compute_bucket_perf(df_filtered_all)

bucket_perf_by_dataset.head(10)

In [ ]:
# %%
bucket_perf_all.head(10)

In [ ]:
# %%
# ----------------------------
# Pareto calculations (filtered)
# ----------------------------
labels_exploded = explode_labels(df_filtered)

# Overall across ALL datasets
overall_all = compute_overall_pareto(labels_exploded)

# Overall by dataset
overall_by_dataset = (
    labels_exploded.groupby(["Dataset", "Labels"])
                   .size().reset_index(name="Count")
                   .sort_values(["Dataset", "Count"], ascending=[True, False])
)
overall_by_dataset["Cumulative %"] = (
    overall_by_dataset.groupby("Dataset")["Count"].transform(lambda x: x.cumsum() / x.sum() * 100)
).round(2)

# By warehouse (Dataset+Warehouse)
by_wh = compute_by_wh_pareto(labels_exploded, WAREHOUSES)

# Combined ALL by warehouse
labels_exploded_all = labels_exploded.copy()
labels_exploded_all["Dataset"] = "ALL"
by_wh_all = compute_by_wh_pareto(labels_exploded_all, WAREHOUSES)

overall_all.head(15)

In [ ]:
# %%
# ----------------------------
# Pareto charts
# ----------------------------
for ds in sorted(by_wh["Dataset"].unique()):
    for wh in sorted(by_wh[by_wh["Dataset"] == ds]["Warehouse"].unique()):
        sub = by_wh[(by_wh["Dataset"] == ds) & (by_wh["Warehouse"] == wh)]
        if not sub.empty:
            plot_pareto(sub, f"Pareto of Labels — {ds} — {wh}", top_n=TOP_N_LABELS)

# Combined ALL for each Warehouse
for wh in sorted(by_wh_all["Warehouse"].unique()):
    sub = by_wh_all[by_wh_all["Warehouse"] == wh]
    if not sub.empty:
        plot_pareto(sub, f"Pareto of Labels — ALL — {wh}", top_n=TOP_N_LABELS)


In [ ]:
# %%
# ----------------------------
# Export for Power BI (filtered)
# ----------------------------
# Merge ALL + dataset-level for overall labels
overall_all_export = overall_all.copy()  # Dataset='ALL'
overall_by_dataset_export = overall_by_dataset.copy()
overall_combined = pd.concat([overall_all_export, overall_by_dataset_export], ignore_index=True)

# By-warehouse combined table: stack ALL + per dataset
by_wh_combined = pd.concat([by_wh, by_wh_all], ignore_index=True)

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    bucket_perf_by_dataset.to_excel(writer, sheet_name="Bucket_Performance", index=False)
    bucket_perf_all.to_excel(writer, sheet_name="Bucket_Performance_ALL", index=False)
    overall_combined.to_excel(writer, sheet_name="Pareto_Labels_Overall", index=False)
    by_wh_combined.to_excel(writer, sheet_name="Pareto_Labels_By_Warehouse", index=False)
    df_filtered.to_excel(writer, sheet_name="Detailed_Data", index=False)

print(f"Exported (filtered) Power BI workbook -> {OUTPUT_XLSX.resolve()}")
